# HW3 b05705017 黃亮穎

## Q1 Classification via Generative Models

In [1]:
import pandas as pd
import pickle
import numpy as np
import itertools as it

with open('phone_train.pickle', 'rb') as fh1:
    traindata = pickle.load(fh1)

with open('phone_test1.pickle', 'rb') as fh2:
    testdata = pickle.load(fh2)
    
x_train = np.array(traindata)[:, :3].astype('float')
y_train = np.array(traindata)[:, 3]
x_test = np.array(testdata)[:, :3].astype('float')

### Q1-1 create mypgc

In [2]:
class mypgc():
    def __init__(self):
        self.typ = np.array(["Phoneonback", "Phoneonfront", "Phoneonleft", "Phoneonright", "Phoneontop", "Phoneonbottom"])
        
    def fit(self, x_trian, y_train):
        
        def mu(typ):
            mu = np.mean(x_train[y_train == typ], axis = 0)
            return mu
        
        def cov(typ):
            return np.cov(x_train[y_train == typ].T)
        
        mean = [[]] * 6
        prior = [0] * 6
        covar = np.zeros((3, 3))
        r = len(y_train)
        
        for i, j in enumerate(self.typ):
            mean[i] = mu(j)
            prior[i] = np.sum(y_train == j) / r
            covar += np.sum(y_train == j) / r * cov(j)
        
        self.mu = mean
        self.cov = covar
        self.prior = prior

    def predict(self, x_test):
        def gaussian(x, mu, cov):
            coef = 1 / ((np.sqrt(2 * np.pi) ** len(x)) * (np.sqrt(np.linalg.det(cov))))
            index = (-1/2) * np.dot((x - mu).T, np.linalg.inv(cov))
            index = np.dot(index, (x - mu))
            prob = coef * np.exp(index)
            return prob
    
        def position(x):
            nume = np.zeros(6)
            deno = 0
            for i in range(len(self.typ)):
                p = self.prior[i] * gaussian(x, self.mu[i], self.cov)
                nume[i] = p
                deno += p
            prob = nume / deno
            idx = np.argsort(prob)
            return self.typ[idx[-1]]
        
        y_pred = np.array([position(i) for i in x_test])
        return y_pred

### Q1-2 train model

In [3]:
pgc1 = mypgc()
pgc1.fit(x_train, y_train)

# calculate and show model parameters
def mu(typ):
    mu = np.mean(x_train[y_train == typ], axis = 0)
    return mu

def cov(typ):
    return np.cov(x_train[y_train == typ].T)

def prec(cov):
    return np.linalg.inv(cov)

def detcov(cov):
    return np.linalg.det(cov)

typp = ["Phoneonback", "Phoneonfront", "Phoneonleft", "Phoneonright", "Phoneontop", "Phoneonbottom"]

model = {}
for i in typp:
     model[i] = {"mu" : mu(i), "cov" : cov(i), "prec" : prec(cov(i)), "detcov" : detcov(cov(i)), "n" : np.sum(y_train == i) , "prior" : np.sum(y_train == i) / len(y_train)}
        
for key, value in model.items():
    print(key, ":")
    for key2, value2 in value.items():
        print("\n", key2, ":\n", value2)
    print()

Phoneonback :

 mu :
 [ 0.20693328 -0.02615235  9.7963762 ]

 cov :
 [[ 0.00299006 -0.00195813  0.00284766]
 [-0.00195813  0.00229887 -0.0024254 ]
 [ 0.00284766 -0.0024254   0.0048876 ]]

 prec :
 [[ 943.63669582  469.54782044 -316.78479596]
 [ 469.54782044 1146.63145078  295.42598486]
 [-316.78479596  295.42598486  535.76812508]]

 detcov :
 5.673178805678628e-09

 n :
 28566

 prior :
 0.17095459523510295

Phoneonfront :

 mu :
 [ 0.11270401  0.14265129 -9.73579633]

 cov :
 [[ 0.00173617 -0.00562249  0.00071319]
 [-0.00562249  0.03069977 -0.00386164]
 [ 0.00071319 -0.00386164  0.00185582]]

 prec :
 [[1415.5956848   258.4845593    -6.15206605]
 [ 258.4845593    91.32078358   90.68682325]
 [  -6.15206605   90.68682325  729.91124637]]

 detcov :
 2.971267016655918e-08

 n :
 29079

 prior :
 0.1740246683064328

Phoneonleft :

 mu :
 [ 9.92639446  0.09275717 -0.01933473]

 cov :
 [[ 0.00192839 -0.00619274  0.00063551]
 [-0.00619274  0.03200844 -0.00317611]
 [ 0.00063551 -0.00317611  0.

### Q1-3 predict test data and show result

In [4]:
ypred = pgc1.predict(x_test)

res = np.array(list(it.zip_longest(ypred[:20], np.array(testdata['label'][:20]))))

print("first 20 predictions and actual positions pair :")
print("|  prediction        |  actual              |")
print(res)

correct = [ypred[i] == testdata['label'][i] for i in range(len(ypred))]
accuracy = float(sum(correct)) / len(ypred)
print()
print("numbers of correct predictions :", sum(correct))
print()
print("accuracy :", "{:.2%}".format(accuracy))


first 20 predictions and actual positions pair :
|  prediction        |  actual              |
[['Phoneonfront' 'Phoneonfront']
 ['Phoneonbottom' 'Phoneonbottom']
 ['Phoneonbottom' 'Phoneonbottom']
 ['Phoneonbottom' 'Phoneonbottom']
 ['Phoneonfront' 'Phoneonfront']
 ['Phoneonright' 'Phoneonright']
 ['Phoneonfront' 'Phoneonfront']
 ['Phoneontop' 'Phoneontop']
 ['Phoneonfront' 'Phoneonfront']
 ['Phoneonfront' 'Phoneonfront']
 ['Phoneonfront' 'Phoneonfront']
 ['Phoneonback' 'Phoneonback']
 ['Phoneonleft' 'Phoneonleft']
 ['Phoneonback' 'Phoneonback']
 ['Phoneonbottom' 'Phoneonbottom']
 ['Phoneonback' 'Phoneonback']
 ['Phoneonright' 'Phoneonright']
 ['Phoneonright' 'Phoneonright']
 ['Phoneontop' 'Phoneontop']
 ['Phoneonleft' 'Phoneonleft']]

numbers of correct predictions : 83511

accuracy : 100.00%


## Q2 Logistic Regression with L2 Regularization

### Q2-1 Data cleaning

In [5]:
data = open("adult.data.txt", 'r')
n = np.array(data.read().splitlines())

test = open("adult.test.txt", 'r')
t = np.array(test.read().splitlines())

dummy = [1, 3, 5, 6, 7, 8, 9, 13]
conti = [0, 2, 4, 10, 11, 12]

dfx = [[j.strip() for j in i.split(',')] for i in n]
dfx = pd.DataFrame(dfx)

for i in dfx.columns:
    idx = dfx[dfx[i] == '?'].index
    dfx.drop(idx, inplace = True)
dfx = dfx.reset_index(drop = True)

for i in dummy:
    num = dfx[i].value_counts()
    mask = dfx[i].isin(num[num > 10].index)
    dfx = dfx[mask]
    dfx.reset_index(drop = True)

y_train = np.array(dfx[14])
dfx = dfx.drop(columns = [14])
y_train[y_train == "<=50K"] = 0
y_train[y_train == ">50K"] = 1
y_train = y_train.astype('float')

categories = {}
for i in dummy:
    dfx[i] = dfx[i].astype('category')
    categories[i] = dfx[i].cat.categories

x_train = pd.get_dummies(dfx, columns = dummy)
x_train = np.array(x_train).astype('float')
dfxt = [[j.strip() for j in i.split(',')] for i in t]
dfxt = pd.DataFrame(dfxt)


for i in dfxt.columns:
    idx = dfxt[dfxt[i] == '?'].index
    dfxt.drop(idx, inplace = True)
dfxt = dfxt.reset_index(drop = True)

y_test = np.array(dfxt[14])
dfxt = dfxt.drop(columns = [14])
y_test[y_test == "<=50K."] = 0
y_test[y_test == ">50K."] = 1
y_test = y_test.astype('float')

for i in dummy:
    dfxt[i] = dfxt[i].astype('category')
    dfxt[i].cat.set_categories(categories[i], inplace = True)

x_test = pd.get_dummies(dfxt, columns = dummy)
x_test = np.array(x_test).astype('float')

mean = np.mean(x_train[:, :6], axis = 0)
std = np.std(x_train[:, :6], axis = 0)
x_train[:, :6] = (x_train[:, :6] - mean) / std
x_test[:, :6] = (x_test[:, :6] - mean) / std

#### summary statistics of data

In [6]:
print("shape of x_train :", x_train.shape)
print("shape of x_test : ", x_test.shape)
print("shape of y_train :", y_train.shape)
print("shape of y_test :", y_test.shape)

shape of x_train : (30152, 102)
shape of x_test :  (15060, 102)
shape of y_train : (30152,)
shape of y_test : (15060,)


In [7]:
df = pd.DataFrame(x_train)
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
count,3.015200e+04,3.015200e+04,3.015200e+04,3.015200e+04,3.015200e+04,3.015200e+04,30152.000000,30152.000000,30152.000000,30152.000000,...,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000
mean,-1.187694e-16,-2.710016e-17,-2.801921e-16,-2.226927e-17,-6.244820e-17,1.861663e-16,0.030976,0.068553,0.739089,0.035620,...,0.001128,0.003615,0.000365,0.002355,0.001393,0.000564,0.000597,0.911880,0.002123,0.000531
std,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,0.173257,0.252696,0.439139,0.185343,...,0.033562,0.060017,0.019097,0.048469,0.037297,0.023738,0.024426,0.283474,0.046023,0.023030
min,-1.632305e+00,-1.666013e+00,-3.576761e+00,-1.474696e-01,-2.184590e-01,-3.333285e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.948575e-01,-6.830421e-01,-4.397050e-01,-1.474696e-01,-2.184590e-01,-7.774463e-02,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,-1.096728e-01,-1.075818e-01,-4.757293e-02,-1.474696e-01,-2.184590e-01,-7.774463e-02,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,6.516436e-01,4.527373e-01,1.128823e+00,-1.474696e-01,-2.184590e-01,3.396324e-01,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,3.925304e+00,1.225606e+01,2.305219e+00,1.335236e+01,1.056266e+01,4.847304e+00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
df2 = pd.DataFrame(x_test)
df2.describe()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
count,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,...,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,0.024953,-0.001658,-0.003360,0.003771,0.001920,0.001690,0.030744,0.068592,0.731806,0.037981,...,0.001859,0.004382,0.000598,0.001992,0.000863,0.000797,0.000531,0.915538,0.001262,0.000465
std,1.018693,0.999622,1.003359,1.039927,1.005553,1.006950,0.172628,0.252768,0.443034,0.191158,...,0.043080,0.066057,0.024440,0.044589,0.029369,0.028218,0.023043,0.278089,0.035498,0.021555
min,-1.632305,-1.668635,-3.576761,-0.147470,-0.218459,-3.333285,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.794857,-0.692221,-0.439705,-0.147470,-0.218459,-0.077745,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,-0.109673,-0.112030,-0.047573,-0.147470,-0.218459,-0.077745,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,0.727775,0.461854,1.128823,-0.147470,-0.218459,0.339632,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,3.925304,12.309959,2.305219,13.352363,9.112309,4.847304,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
class mylogistic_l2:
    def __init__(self, reg_vec, max_iter, tol, add_intercept):
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept    
        
    def fit(self, x_train, y_train, verbose = False):
        
        if self.add_intercept:
            self.x_train = np.c_[x_train, np.ones(x_train.shape[0])]
        else:
            self.x_train = x_train
        
        self.y_train = y_train
        
        w = np.linalg.inv(np.dot(self.x_train.T, self.x_train) + np.mean(self.reg_vec) * np.identity(self.reg_vec.shape[0]))
        w = np.dot(w, self.x_train.T)
        w = np.dot(w, self.y_train)
        self.w = w
        e = self.reg_error(self.x_train, self.y_train, self.w, self.reg_vec)
        
        for i in range(self.max_iter):
            z = np.dot(self.x_train, self.w)
            pred = self.sigmoid(z)
            error = pred - self.y_train
            grad = (self.reg_vec * self.w) + np.dot(self.x_train.T, error)
            R = pred * (1 - pred)
            hess = self.x_train.T * R
            hess = np.dot(hess, self.x_train) + np.identity(self.reg_vec.shape[0]) * self.reg_vec
            if i == 0 and verbose:
                print("gradient :", grad)
                print("hessian :", hess)
            self.w = self.w - np.dot(np.linalg.pinv(hess), grad)
            
            next_error = self.reg_error(self.x_train, self.y_train, self.w, self.reg_vec)
            if abs(e - next_error) < self.tol:
                break
            e = next_error
            
    def predict(self, x_test):
        if self.add_intercept:
            x_test = np.c_[x_test, np.ones(x_test.shape[0])]
        z = np.dot(x_test, self.w)
        res = np.array([self.sigmoid(i) for i in z])
        res[res >= 0.5] = 1
        res[res < 0.5] = 0
        return res
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def reg_error(self, x_train, y_train, w, reg_vec):
        z = np.dot(x_train, w) 
        reg_term = (1/2) * np.dot(reg_vec * w.T, w)
        return -1 * np.sum((y_train * np.log(self.sigmoid(z) + 1e-5)) + ((1 - y_train) * np.log(1 - self.sigmoid(z) + 1e-5))) + reg_term

### Q2-2 Derive the gradient and hession matrix for the new E(w)

In [10]:
lambda_vec = np.array([1] * (x_train.shape[1] + 1))
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train, True)

gradient : [-2.39097995e+03  8.86325719e+01 -3.32606519e+03 -2.25982859e+03
 -1.49120297e+03 -2.27153516e+03  1.91715719e+02  5.73307313e+02
  7.45742056e+03  8.02662888e+01  7.09574275e+02  3.79304697e+02
  6.91263270e+00  3.65511509e+02  4.79521522e+02  1.66132495e+02
  7.10235693e+01  1.34989900e+02  2.52152806e+02  2.08660882e+02
  3.10885150e+02  3.94138412e+02  9.12643019e+02 -2.63477247e+01
  3.69979223e+03  1.16733475e+02  2.25693199e+01 -3.98688825e+01
  2.32996381e+03  1.76670691e+03  3.06424960e+00  2.18847874e+03
  1.61627191e+02  4.50554470e+03  4.19771023e+02  3.53308682e+02
  1.48508201e+03  1.33128508e+03  5.27277285e+02  4.07853917e+02
  6.12445983e+02  7.98083241e+02  1.50637200e+03  7.07478380e+01
  6.44603724e+02  1.63657457e+02  1.05898674e+03  2.46244066e+02
  5.45862140e+02  1.93281196e+03  3.24131972e+03  4.16617237e+02
  2.18423393e+03  1.44565502e+03  1.77863620e+02  1.16865205e+02
  2.59809092e+02  1.13181834e+03  9.95511842e+01  7.79045766e+03
  4.05062335e+

### Q2-3 Case 1

In [11]:
p = logic1.predict(x_test)
print("w :")
print(logic1.w)
print("\naccuracy :", "{:.4%}".format(sum(p == y_test) / len(y_test)))

w :
[ 3.33300239e-01  7.91950118e-02  7.50781031e-01  2.33387540e+00
  2.58016680e-01  3.52920655e-01  5.03917484e-01 -1.87836061e-01
  1.95888083e-03  1.77435295e-01 -4.85830477e-01 -3.09824864e-01
 -1.02513995e+00 -7.37006767e-04 -1.96244035e-01 -1.34587849e-01
  6.26654377e-01  4.48135863e-01  2.46645745e-02  4.68906082e-02
 -4.90177174e-01 -2.02164859e-01 -1.62129305e-01 -1.61301013e-02
 -1.10533240e-01 -9.86066332e-02 -1.17390699e+00  1.82003137e-01
 -6.84510520e-02 -7.20124646e-01  1.45016439e+00  1.16040810e+00
 -6.78265843e-01 -1.20885947e+00 -7.96290058e-01 -5.32352175e-01
  1.83346713e-02  8.34528999e-02  8.19526649e-01 -9.60845325e-01
 -6.63925233e-01 -2.43581800e-01 -7.94506650e-01 -1.63580895e+00
  5.32751247e-01  6.06061717e-01  3.10770631e-01  6.74052130e-01
 -7.16016722e-02 -2.58776529e-01 -2.92864853e-02 -7.93917742e-01
 -1.16056471e+00 -1.53654181e-01  1.07087996e+00 -6.25219676e-01
  1.19552636e-01 -2.28357231e-01 -5.14768775e-01 -7.65266427e-02
 -1.08972678e+00 -2.3

### Q2-3 Case 2

In [12]:
lambda_vec = np.array([1] * (x_train.shape[1]) + [0])
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)

In [13]:
p = logic1.predict(x_test)
print("w :")
print(logic1.w)
print("\naccuracy :", "{:.4%}".format(sum(p == y_test) / len(y_test)))

w :
[ 0.33364317  0.07921947  0.73771785  2.33349074  0.25802921  0.35304508
  0.70874053  0.01734641  0.20829646  0.38214911 -0.28024988 -0.10512748
 -0.93115516  0.07202585 -0.11834194 -0.05311322  0.67152053  0.50324191
  0.08716597  0.11338255 -0.38546955 -0.10254933 -0.05190874  0.10710446
 -0.02050343  0.01627788 -1.16578489  0.30036517  0.02658678 -0.52710016
  1.61481294  1.36915804 -0.49315002 -1.01479456 -0.60643628 -0.34248996
  0.12439724  0.18891847  0.92533953 -0.8562578  -0.55974627 -0.13823864
 -0.68865402 -1.57553304  0.63935265  0.71143584  0.416165    0.77915193
  0.03366911 -0.04466318  0.20015783 -0.57913369 -0.93735843  0.07576444
  1.28523303 -0.36713728  0.39299715  0.04223838 -0.26282764  0.1947294
 -0.42725385  0.42725385  1.00666429  0.50194614 -0.45741932 -1.24076572
  0.52725371 -0.86931547 -0.02818841 -0.31476459  0.47306518  0.62939891
  0.62362559 -0.58679235 -0.03048527  0.12377705 -0.14381483  0.02438259
  0.06221836 -0.24850166  0.19431027  0.52579686

### Q2-3 Case 3

In [14]:
lambda_vec = np.array([1] * 6 + [0.5] * (x_train.shape[1] - 6) + [0])
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)

In [15]:
p = logic1.predict(x_test)
print("w :")
print(logic1.w)
print("\naccuracy :", "{:.4%}".format(sum(p == y_test) / len(y_test)))

w :
[ 0.33420362  0.07934715  0.78245531  2.33563809  0.25820902  0.35331032
  0.76974011  0.07589062  0.26756247  0.4425935  -0.22119325 -0.04686142
 -1.28773203  0.18176537 -0.02660347  0.01937259  0.8977025   0.68435782
  0.23217308  0.24428472 -0.38422598 -0.08085138 -0.06537975  0.04502774
  0.03694934 -0.01388755 -2.09391972  0.25717122  0.06606347 -0.57253724
  1.82595484  1.39795845 -0.54771851 -1.05883542 -0.65626908 -0.38855304
  0.16435265  0.2285428   0.96654543 -0.82320513 -0.52463501 -0.09825492
 -0.65394918 -2.04836557  0.67978665  0.75553079  0.45687204  0.82328825
  0.0734912  -0.08585855  0.23338451 -0.58848398 -0.9231019   0.11168247
  1.25237746 -0.37878867  0.4116955   0.04056691 -0.26545931  0.19198557
 -0.42917962  0.42917962  1.18537608  0.55010209 -0.47590613 -1.45953272
  0.58133206 -1.06413979 -0.01038468 -0.31799658  0.52415315  0.7297059
  0.67380915 -0.63657862 -0.01087692  0.17268524 -0.23664217  0.03733919
  0.10100075 -0.24715609  0.23741301  0.64150988

### Q2-4 search a1, a2

In [16]:
from sklearn.model_selection import train_test_split

x_train_s, x_tune, y_train_s, y_tune = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

max_acc = 0
alpha = 0
grids = np.logspace(-2, 2, 10)
for i in grids:
    lambda_vec = np.array([i] * (x_train.shape[1]) + [0])
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_train_s, y_train_s, False)
    p = logic1.predict(x_tune)
    acc = sum(p == y_tune) / len(y_tune)
    if acc >= max_acc:
        max_acc = acc
        alpha = i
print("a1*, a2* :", alpha)
print("accuracy :", "{:.4%}".format(max_acc))

a1*, a2* : 12.915496650148826
accuracy : 85.5438%


### Q2-4 search new a2

In [17]:
alpha2 = 0
max_acc2 = 0
for i in grids:
    lambda_vec = np.array([alpha] * 6 + [i] * (x_train.shape[1] - 6) + [0])
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_train_s, y_train_s, False)
    p = logic1.predict(x_tune)
    acc = sum(p == y_tune) / len(y_tune)
    if acc >= max_acc2:
        max_acc2 = acc
        alpha2 = i
print("a2* :", alpha2)
print("accuracy :", "{:.2%}".format(max_acc2))

a2* : 4.6415888336127775
accuracy : 85.58%


### Q2-4 search new a1

In [18]:
alpha1 = 0
max_acc1 = 0
for i in grids:
    lambda_vec = np.array([i] * 6 + [alpha2] * (x_train.shape[1] - 6) + [0])
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_train_s, y_train_s, False)
    p = logic1.predict(x_tune)
    acc = sum(p == y_tune) / len(y_tune)
    if acc >= max_acc1:
        max_acc1 = acc
        alpha1 = i
print("a1* :", alpha1)
print("accuracy :", "{:.4%}".format(max_acc1))

a1* : 12.915496650148826
accuracy : 85.5769%


### Q2-4 report a1, a2

In [19]:
print("best a1 :", alpha1)
print("best a2 :", alpha2)

best a1 : 12.915496650148826
best a2 : 4.6415888336127775


### Q2-4 train model

In [20]:
lambda_vec = np.array([alpha1] * 6 + [alpha2] * (x_train.shape[1] - 6) + [0])
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train, False)
p = logic1.predict(x_test)
acc = sum(p == y_test) / len(y_test)
print("accuracy :", "{:.4%}".format(acc))

accuracy : 84.8074%


### Q2-5 

In [21]:
from sklearn.linear_model import LogisticRegression
max_acc = 0
best_c = 0
for c in grids:
    logic2 = LogisticRegression(solver = 'lbfgs', max_iter = 1000, tol = 1e-5, n_jobs = -1, C = c)
    logic2.fit(x_train_s, y_train_s)
    p = logic2.predict(x_tune)
    accc = sum(p == y_tune) / len(y_tune)
    if accc > max_acc:
        max_acc = accc
        best_c = c
        
logic2 = LogisticRegression(solver = 'lbfgs', max_iter = 1000, tol = 1e-5, n_jobs = -1, C = best_c)
logic2.fit(x_train, y_train)
p = logic2.predict(x_test)
acc2 = sum(p == y_test) / len(y_test)

print("====== scikit learn logistic regression model result ======")
print("w :")
print(logic2.coef_)
print("\naccuracy :", "{:.4%}".format(acc2))

print("\n====== my own logistic regression model result ======")
print("w :")
print(logic1.w)
print("\naccuracy :", "{:.4%}".format(acc))

print("\naccuracy difference between two models :", abs(acc - acc2))

====== scikit learn logistic regression model result ======
w :
[[ 3.33032097e-01  7.79814273e-02  6.92675553e-01  2.16601256e+00
   2.53795679e-01  3.48928264e-01  5.36557610e-01 -9.23816497e-02
   7.60354219e-02  2.40070549e-01 -4.00265480e-01 -2.02553538e-01
  -1.51723129e-01 -1.68347521e-02 -1.57164065e-01 -7.29630030e-02
   1.49252378e-01  1.57764520e-01 -3.08548222e-02 -1.44652703e-02
  -3.10075462e-01 -8.27678192e-02  4.75597725e-04  1.48484483e-01
  -5.07572586e-02  7.22570423e-02 -1.16220104e-01  3.09621546e-01
   1.99867719e-02 -2.86224348e-01  4.16550872e-01  1.25528953e+00
  -1.84519022e-01 -7.70352401e-01 -3.06964196e-01 -1.18040652e-01
   1.37662516e-02  7.34688638e-02  7.84722334e-01 -8.35258541e-01
  -5.68510500e-01 -2.34601054e-01 -7.16818268e-01 -3.39148763e-01
   5.07934386e-01  5.14180175e-01  2.86321788e-01  5.93590782e-01
  -7.39076710e-02  1.41424449e-01  1.83145686e-02 -4.35890569e-01
  -8.93595699e-01 -1.32109716e-01  1.30759675e+00 -2.34456344e-01
   2.1914460

When random state is set to 42, two models have same prediction ability.